In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData


import psycopg2
import psycopg2.extras
import json
import os
import geopandas as gpd

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

ModuleNotFoundError: No module named 'geopandas'

In [18]:
db, conn = pgconnect(credentials)

Connected successfully.


In [19]:
def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [20]:
def split(word):
    return [char for char in word]
def toInt(st):
    parts=split(st)
    if parts.count(",")>0:
        parts.remove(",")
    i=0
    num=0
    while i<len(parts):
        num+=10**(len(parts)-i-1)* int(parts[i])
        i+=1
    return num

In [21]:
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

In [22]:
conn.execute("""
DROP TABLE IF EXISTS BusinessStats;
CREATE TABLE BusinessStats(
   area_id INTEGER,
   area_name VARCHAR(50),
   number_of_businesses INTEGER,
   accommodation_and_food_services INTEGER,
   retail_trade INTEGER,
   agriculture_forestry_and_fishing INTEGER,
   health_care_and_social_assistance INTEGER,
   public_administration_and_safety INTEGER,
   transport_postal_and_warehousing INTEGER
);""")

In [23]:
Businessdata = pd.read_csv('BusinessStats.csv')
Businessdata.to_sql("businessstats", con=conn, if_exists='append', index=False)

In [24]:
SA2_data = gpd.read_file("SA2_2016_AUST/SA2_2016_AUST.shp")
SA2_data = SA2_data.replace(to_replace='None', value=np.nan).dropna()
SA2_data

NameError: name 'gpd' is not defined